<h2>Photodiode Analysis Code</h2>

Ansley Kunnath

Updated 04/15/24

In [2]:
# Load data
########## Run with Python 3.9.12 (for Ansley)

import mne
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("TkAgg")
import scipy.stats as stats
#json import needed
import json

########## You may need to change the path and file name:
eeg_path = "/Users/andrewkim/Desktop/PhotodiodeData/"  
file_name = "Kim_Andrew_2024-07-03_17-58-39"
file_eeg = eeg_path + file_name + ".eeg"
file_vhdr = eeg_path + file_name + ".vhdr"
file_vmrk = eeg_path + file_name + ".vmrk"

# Load and plot the raw data
raw = mne.io.read_raw_brainvision(file_vhdr)
events, event_id = mne.events_from_annotations(raw)

#raw.crop(tmin=22, tmax=190)
#raw.plot()

Extracting parameters from /Users/andrewkim/Desktop/PhotodiodeData/Kim_Andrew_2024-07-03_17-58-39.vhdr...
Setting channel info structure...
Used Annotations descriptions: ['Marker/Impedance', 'New Segment/', 'Stimulus/s1', 'Stimulus/s2', 'Stimulus/s3', 'Stimulus/s5']


/var/folders/m2/8_k5938s5_l8bzxcjsy3qyk00000gn/T/ipykernel_51485/745755210.py:20: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_brainvision(file_vhdr)


In [3]:
# Create epochs for checkerboard events

stimulus_s1_events = events[events[:, 2] == event_id['Stimulus/s1']]
stimulus_s1_events, event_id
tmin, tmax = 0, 0.250  
epochs = mne.Epochs(raw, events=stimulus_s1_events, event_id=event_id['Stimulus/s1'],
                    tmin=tmin, tmax=tmax, baseline=None, preload=True)
epochs = epochs.pick_channels(['BIP3'])

#epochs.plot() 

Not setting metadata
105 matching events found
No baseline correction applied
0 projection items activated
Loading data for 105 events and 1001 original time points ...
0 bad epochs dropped
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


In [4]:
# Calculate the first latency that exceeds # of MADs for each epoch 

########## CHANGE MAD_FACTOR
mad_factor = 4 # or 2 to be less strict

median_amplitude = []
first_time = []
peak_latencies = []

for index, epoch in enumerate(epochs.get_data()):
    positive_epoch = abs(epoch)*1000
    median_amplitude = np.median(positive_epoch)
    mad = np.median(np.abs(positive_epoch - median_amplitude))
    threshold = median_amplitude + (mad_factor * mad)
    exceed_index = np.argmax(positive_epoch > threshold)
    if exceed_index > 0:
        first_time = epochs.times[exceed_index] 
    else:
        first_time = None 
        print(f"No exceedance found in epoch {index}") 
    peak_latencies.append(first_time)

# Calculate latencies in milliseconds
peak_latencies_ms = np.array([lat * 1000 if lat is not None else None for lat in peak_latencies])
valid_latencies = peak_latencies_ms[peak_latencies_ms != np.array(None)]
total_events = len(valid_latencies)

# Calculate latency CI in milliseconds
ncies_ms = np.array([lat * 1000 if lat is not None else None for lat in peak_latencies])
valid_latencies = peak_latencies_ms[peak_latencies_ms != np.array(None)]

# Calculate average and confidence interval only for valid latencies
if len(valid_latencies) > 0:
    median_latency_ms = np.median(valid_latencies)
    sem_latency = stats.sem(valid_latencies)  # SEM = std / sqrt(n)
    confidence_level = 0.95
    ci_width = sem_latency * stats.t.ppf((1 + confidence_level) / 2, len(valid_latencies) - 1)
    confidence_interval = (median_latency_ms - ci_width, median_latency_ms + ci_width)
else:
    median_latency_ms = None
    confidence_interval = (None, None)

print("")
print(f"Median Latency: {median_latency_ms:.0f} ms")
print(f"95% Confidence Interval: ({confidence_interval[0]:.3f}, {confidence_interval[1]:.3f})")
print(f"Total Events: {total_events} out of 100")

#save the latency values separately
peak_latencies_ms = [lat * 1000 if lat is not None else None for lat in peak_latencies]
with open('photodiode_latencies.json', 'w') as f:
    json.dump(peak_latencies_ms, f)

print("Photodiode latencies saved to photodiode_latencies.json")

No exceedance found in epoch 10
No exceedance found in epoch 20
No exceedance found in epoch 24
No exceedance found in epoch 28
No exceedance found in epoch 30
No exceedance found in epoch 40
No exceedance found in epoch 41
No exceedance found in epoch 43
No exceedance found in epoch 48
No exceedance found in epoch 50
No exceedance found in epoch 57
No exceedance found in epoch 60
No exceedance found in epoch 70
No exceedance found in epoch 80
No exceedance found in epoch 90
No exceedance found in epoch 100
No exceedance found in epoch 102

Median Latency: 60 ms
95% Confidence Interval: (47.837, 71.163)
Total Events: 88 out of 100
Photodiode latencies saved to photodiode_latencies.json


/var/folders/m2/8_k5938s5_l8bzxcjsy3qyk00000gn/T/ipykernel_51485/4145922955.py:10: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  for index, epoch in enumerate(epochs.get_data()):


In [5]:
# Plot individual epochs

########## CHANGE X_VALUES BASED ON WHICH EPOCHS DID NOT EXCEED THE THRESHOLD
x_values = [1, 2, 3, 4, 5]

fig, axes = plt.subplots(len(x_values), 1, figsize=(10, 10), sharex=True, sharey=False)
for i, x in enumerate(x_values):
    epoch = epochs.get_data()[x]
    abs_epoch = abs(epoch[0]) * 1000 
    median_amplitude = np.median(abs_epoch)
    mad = np.median(np.abs(abs_epoch - median_amplitude))
    threshold = median_amplitude + (mad_factor * mad)
    times_in_ms = epochs.times * 1000
    exceed_index = np.argmax(abs_epoch > threshold)
    if exceed_index > 0:
        first_time = epochs.times[exceed_index] * 1000 
    else:
        first_time = None  # No point exceeded the threshold
    
    min_time = min(times_in_ms)
    max_time = max(times_in_ms)
    vertical_lines = np.arange(min_time, max_time, 2)

    axes[i].plot(times_in_ms, abs_epoch)
    axes[i].axvline(x=first_time if first_time is not None else 0, color='black', 
        label=f"Screen Change: {first_time:.2f} ms" if first_time is not None else "Screen Change: None")
    axes[i].axhline(y=threshold, color='r', linestyle='--', label=f"Threshold: {threshold:.2f} mV")
    axes[i].axhline(y=median_amplitude, color='b', linestyle='--', label=f"Median: {median_amplitude:.2f} mV")
    for line in vertical_lines:
        axes[i].axvline(x=line, color='gray', linestyle='--', linewidth=0.5, alpha=0.5)
    axes[i].set_title(f"Epoch {x}")
    axes[i].legend()

plt.xlabel("Time (ms)")
plt.ylabel("Amplitude (mV)")
plt.savefig('Plot Epochs.png')
plt.show()


/var/folders/m2/8_k5938s5_l8bzxcjsy3qyk00000gn/T/ipykernel_51485/2414058590.py:8: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epoch = epochs.get_data()[x]
/var/folders/m2/8_k5938s5_l8bzxcjsy3qyk00000gn/T/ipykernel_51485/2414058590.py:8: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epoch = epochs.get_data()[x]
/var/folders/m2/8_k5938s5_l8bzxcjsy3qyk00000gn/T/ipykernel_51485/2414058590.py:8: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epoch = epochs.get_data()[x]
/var/folders/m2/8_k5938s5_l8bzxcjsy3qyk00000gn/T/ipykernel_51485/2414058590.py:8: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epoch = epochs.get_data()[x]


In [6]:
# Histogram of latency distributions

########## SET BIN SIZE
num_bins = 6 

plt.figure(figsize=(10, 6))
bin_edges = np.linspace(min(valid_latencies), max(valid_latencies), num_bins + 1)
rounded_bin_edges = np.round(bin_edges)
n, bins, patches = plt.hist(valid_latencies, bins=bin_edges, edgecolor='black', linewidth=1.5)
plt.xticks(rounded_bin_edges)
for count, x in zip(n, bins[:-1]):
    plt.text(x + (bins[1]-bins[0])/2, count, str(int(count)), ha='center', va='bottom')
plt.title('Histogram of Latencies')
plt.xlabel('Latency (ms)')
plt.ylabel('Frequency')
plt.savefig('Histogram.png')
plt.show()


invalid command name "6268111808process_stream_events"
    while executing
"6268111808process_stream_events"
    ("after" script)
can't invoke "event" command:  application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"


In [7]:
# Scatter plot of the latencies

plt.figure(figsize=(10, 6))
plt.scatter(range(len(valid_latencies)), valid_latencies, color='black')
plt.axhline(median_latency_ms, color='red', linestyle='dashed', linewidth=2)
plt.title('Scatter Plot of Latencies')
plt.xlabel('Event')
plt.ylabel('Latency (ms)')
plt.grid(True)
plt.savefig('Scatter Plot.png')
plt.show()


invalid command name "6263422848process_stream_events"
    while executing
"6263422848process_stream_events"
    ("after" script)
can't invoke "event" command:  application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"


In [8]:
import mne
import numpy as np
from mne.preprocessing import (ICA)
from autoreject import AutoReject
import matplotlib
from autoreject import AutoReject

#for storing and exchanging data
import json
matplotlib.use("TkAgg")

#load the photodiode latency data
with open('photodiode_latencies.json', 'r') as f:
    photodiode_latencies = json.load(f)

# Adjust epoch start times based on photodiode latencies
adjusted_events = []
for event, latency in zip(events, photodiode_latencies):
    if latency is not None:
        adjusted_event = event.copy()
        adjusted_event[0] += int(latency)  # Adjusting event start time by photodiode latency
        adjusted_events.append(adjusted_event)

adjusted_events = np.array(adjusted_events)

# Create epochs with adjusted events
tmin, tmax = 0, 0.250
epochs = mne.Epochs(raw, events=adjusted_events, event_id=event_id['Stimulus/s1'], tmin=tmin, tmax=tmax, baseline=None, preload=True)

# Continue with VEP analysis as usual
evoked = epochs.average()
evoked.plot()

Not setting metadata
33 matching events found
No baseline correction applied
0 projection items activated
Loading data for 33 events and 1001 original time points ...
0 bad epochs dropped
Need more than one channel to make topography for eeg. Disabling interactivity.


invalid command name "13056630912process_stream_events"
    while executing
"13056630912process_stream_events"
    ("after" script)
can't invoke "event" command:  application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"


<Figure size 640x300 with 1 Axes>

In [9]:
eeg_path = "C://Users//neuro//Documents//VEP//VEP Data//"  # You will need to change this location
file_name = "AJK-03-01-24"
file_eeg = eeg_path + file_name + ".eeg"
file_vhdr = eeg_path + file_name + ".vhdr"
file_vmrk = eeg_path + file_name + ".vmrk"

In [10]:
raw = mne.io.read_raw_brainvision(file_vhdr)
#drop_channels = ['BIP1','BIP2','EOG','TEMP1','ACC1','ACC2','ACC3']
#raw = raw.drop_channels(drop_channels)
#raw.plot()
events_from_annot, event_dict = mne.events_from_annotations(raw)
del event_dict['Stimulus/s5']

Extracting parameters from C://Users//neuro//Documents//VEP//VEP Data//AJK-03-01-24.vhdr...


FileNotFoundError: [Errno 2] No such file or directory: '/Users/andrewkim/Downloads/C:/Users/neuro/Documents/VEP/VEP Data/AJK-03-01-24.vhdr'

In [ ]:
highpass = 1
lowpass = 20
notch = 60

raw_filtered = raw.load_data().filter(highpass, lowpass).notch_filter(np.arange(notch, (notch * 3), notch))
#raw_filtered = raw.resample(resample).filter(highpass, lowpass).notch_filter(np.arange(notch, (notch * 3), notch))

eeg_1020 = raw_filtered.copy().set_eeg_reference(ref_channels = 'average') # ref_channels='['Fz']'
ten_twenty_montage = mne.channels.make_standard_montage('standard_1020')
eeg_1020 = eeg_1020.set_montage(ten_twenty_montage, on_missing = 'ignore')
#del raw, raw_filtered, ten_twenty_montage
eeg_1020.info['bads'] = []
picks = mne.pick_types(eeg_1020.info, meg=False, eeg=True, stim=False, eog=False, include=[], exclude=[])

epochs = mne.Epochs(eeg_1020,
                    events=events_from_annot,
                    event_id=event_dict,
                    tmin=-0.050,
                    tmax=0.500,   #duration of stimulus or response
                    baseline=None,
                    reject=None,
                    verbose=False,
                    preload=True,
                    detrend=None,
                    event_repeated='drop')


In [ ]:
n_interpolates = np.array([1, 4, 32])
consensus_percs = np.linspace(0, 1.0, 11)
ar = AutoReject(n_interpolates,
                consensus_percs,
                picks=picks,
                thresh_method='random_search',
                random_state=42)    #random n state
epochs_ar = ar.fit_transform(epochs);


In [ ]:
ica = ICA(n_components = 16, max_iter = 'auto', random_state = 123)
ica.fit(epochs_ar)

ica_z_thresh = 1.96
epochs_clean = epochs_ar.copy()
eog_indices, eog_scores = ica.find_bads_eog(epochs_clean,
                                            ch_name=['Fp1', 'F8'],
                                            threshold=ica_z_thresh)
ica.exclude = eog_indices
ica.plot_scores(eog_scores)
ica.plot_sources(epochs_ar)
ica.plot_components()
print(eog_indices)
ica.apply(epochs_clean)
epochs_final = epochs_clean.copy()
#del eeg_1020, epochs, epochs_ar, eog_indices, eog_scores


In [ ]:
baseline_tmin, baseline_tmax = -0.05, 0
baseline = (baseline_tmin, baseline_tmax)

VEP = epochs_final['Stimulus/s1'].apply_baseline(baseline).average()
blank = epochs_final['Stimulus/s2'].apply_baseline(baseline).average()

fig = mne.viz.plot_compare_evokeds(VEP, picks=['Oz','O1','O2'], combine="mean", show=False, time_unit="ms")
fig[0].savefig("VEP Data/"+file_name+"-VEP_Occipital")

fig = mne.viz.plot_compare_evokeds(VEP, picks=['O1','O2'], combine="mean", show=False, time_unit="ms")
fig[0].savefig("VEP Data/"+file_name+"-VEP_O1_O2")

fig = mne.viz.plot_compare_evokeds(VEP, picks=['Oz'], show=False, time_unit="ms")
fig[0].savefig("VEP Data/"+file_name+"-VEP_Oz")

In [ ]:
evokeds = dict(
    Checkerboard=list(epochs_final['Stimulus/s1'].iter_evoked()),
    Blank=list(epochs_final['Stimulus/s2'].iter_evoked()),
)

fig = mne.viz.plot_compare_evokeds(evokeds, 
                                   colors=dict(Checkerboard="red", Blank="black"), 
                                   ci=False, #0.95
                                   picks=['Oz','O1','O2'], time_unit="ms", combine="mean")
fig[0].savefig("VEP Data/"+file_name+"-Compare_Stimuli")